In [90]:
import os
import glob
import pandas as pd
import numpy as np
import re

In [76]:
# Combine CSV
# source: https://www.freecodecamp.org/news/how-to-combine-multiple-csv-files-with-8-lines-of-code-265183e0854/

'''
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
combined_csv.to_csv("INI_Holder_Detail_Combined.csv", index=False, encoding='utf-8-sig')
'''

'\nextension = \'csv\'\nall_filenames = [i for i in glob.glob(\'*.{}\'.format(extension))]\n\ncombined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])\ncombined_csv.to_csv("INI_Holder_Detail_Combined.csv", index=False, encoding=\'utf-8-sig\')\n'

In [99]:
df = pd.read_csv('INI_Holder_Detail_Combined.csv')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (8,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [245]:
df_names = pd.read_csv('合资金融机构.csv')

In [246]:
names = df_names[['机构简称','外方机构基金名称']]
names['count'] = 0
names

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,机构简称,外方机构基金名称,count
0,高盛高华,高盛,0
1,摩根士丹利华鑫,摩根士丹利,0
2,瑞银,瑞银,0
3,瑞信,瑞信,0
4,中德,中德,0
...,...,...,...
77,首奕,首奕,0
78,韩华,韩华,0
79,润晖,润晖,0
80,荷宝,荷宝,0


In [79]:
len(df)

5281899

In [80]:
# Calculate levenshtein ratio & distance
# source: https://www.datacamp.com/community/tutorials/fuzzy-string-python
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [227]:
# Use 100,000 data points as test set
df_test = df.loc[0:100000]

In [228]:
# Method 1: Check levenshtein_ratio and use regular expression to see if ShareHolderName contains institution name  
for i in range(len(names)):
    # Loop through institution name
    name = names.iloc[i]['外方机构基金名称']
    print(name, i)
    count = 0
    
    for j in range(len(df_test)):
        # Loop through ShareHolderName and check if it has a ratio greater than 0.2 and contains institution name
        shareholdername = df_test.iloc[j]['ShareHolderName']
        temp = levenshtein_ratio_and_distance(name, str(shareholdername),ratio_calc = True)
    
        if temp > 0.2:
            if re.search(name, shareholdername):
                count += 1
    print(count)

高盛 0
0
摩根士丹利 1
695
瑞银 2
110
瑞信 3
525
中德 4
0
申港 5
0
华兴 6
36
汇丰 7
163
东亚 8
44
野村 9
0
摩根大通 10
0
统一 11
0
大和 12
0
星展 13
0
国泰 14
614
鹏华 15
787
嘉实 16
655
长盛 17
520
富国 18
584
融通 19
362
国投瑞银 20
749
泰达宏利 21
845
华宝 22
685
国联安 23
367
海富通 24
725
景顺长城 25
1360
兴全 26
81
申万菱信 27
582
中海 28
485
上投摩根 29
752
光大保德信 30
635
富兰克林国海 31
492
华泰柏瑞 32
1574
工银瑞信 33
1599
交银施罗德 34
533
中信保诚 35
72
建信 36
519
汇丰晋信 37
270
中邮 38
100
信达澳银 39
226
中欧 40
152
浦银安盛 41
543
农银汇理 42
889
民生加银 43
451
方正富邦 44
149
华宸未来 45
50
华润元大 46
57
中加 47
44
国开泰富 48
3
永赢 49
54
圆信永丰 50
34
恒生前海 51
15
富达 52
15
瑞银 53
110
英仕曼 54
0
富敦 55
0
惠理 56
0
景顺纵横 57
0
路博迈 58
0
安本 59
13
施罗德 60
450
贝莱德 61
0
安中 62
109
桥水 63
0
元胜 64
0
毕盛 65
0
瀚亚 66
2
未来益财 67
0
联博汇智 68
0
安联 69
2
德劭 70
0
罗素 71
0
弘收 72
0
威廉欧奈尔 73
0
鲍尔赛嘉 74
0
迈德瑞 75
0
柏基 76
0
首奕 77
0
韩华 78
0
润晖 79
0
荷宝 80
0
澳帝桦 81
0


In [229]:
# Method 2: Only check if ShareHolderName contains institution name
for i in range(len(names)):
    name = names.iloc[i]['外方机构基金名称']
    print(name, i)
    count = 0
    
    for j in range(len(df_test)):
        if re.search(name, df_test.iloc[j]['ShareHolderName']):
            count += 1
    print(count)  

高盛 0
1
摩根士丹利 1
695
瑞银 2
749
瑞信 3
1604
中德 4
0
申港 5
0
华兴 6
161
汇丰 7
405
东亚 8
46
野村 9
0
摩根大通 10
0
统一 11
0
大和 12
1
星展 13
0
国泰 14
1223
鹏华 15
2355
嘉实 16
2515
长盛 17
1471
富国 18
1667
融通 19
633
国投瑞银 20
749
泰达宏利 21
848
华宝 22
1388
国联安 23
386
海富通 24
731
景顺长城 25
1360
兴全 26
315
申万菱信 27
589
中海 28
611
上投摩根 29
754
光大保德信 30
635
富兰克林国海 31
492
华泰柏瑞 32
1597
工银瑞信 33
1599
交银施罗德 34
533
中信保诚 35
72
建信 36
1414
汇丰晋信 37
270
中邮 38
572
信达澳银 39
226
中欧 40
682
浦银安盛 41
543
农银汇理 42
889
民生加银 43
451
方正富邦 44
149
华宸未来 45
50
华润元大 46
57
中加 47
81
国开泰富 48
3
永赢 49
199
圆信永丰 50
34
恒生前海 51
15
富达 52
18
瑞银 53
749
英仕曼 54
0
富敦 55
0
惠理 56
1
景顺纵横 57
0
路博迈 58
0
安本 59
13
施罗德 60
535
贝莱德 61
0
安中 62
736
桥水 63
0
元胜 64
0
毕盛 65
0
瀚亚 66
2
未来益财 67
0
联博汇智 68
0
安联 69
2
德劭 70
0
罗素 71
0
弘收 72
0
威廉欧奈尔 73
0
鲍尔赛嘉 74
0
迈德瑞 75
0
柏基 76
0
首奕 77
0
韩华 78
0
润晖 79
0
荷宝 80
0
澳帝桦 81
0


In [247]:
# Method 3: Only check levenshtein_ratio
for i in range(len(names)):
    name = names.iloc[i]['外方机构基金名称']
    print(name, i)
    count = (df_test['ShareHolderName'].apply(lambda x: levenshtein_ratio_and_distance(name, x, True)) > 0.2).sum()
    names.at[i,'count'] = count
    print(count)

高盛 0
1
摩根士丹利 1
767
瑞银 2
150
瑞信 3
549
中德 4
44
申港 5
8
华兴 6
253
汇丰 7
247
东亚 8
76
野村 9
0
摩根大通 10
126
统一 11
2
大和 12
61
星展 13
3
国泰 14
680
鹏华 15
853
嘉实 16
683
长盛 17
527
富国 18
720
融通 19
365
国投瑞银 20
2296
泰达宏利 21
951
华宝 22
792
国联安 23
551
海富通 24
1027
景顺长城 25
1675
兴全 26
107
申万菱信 27
729
中海 28
743
上投摩根 29
2147
光大保德信 30
738
富兰克林国海 31
527
华泰柏瑞 32
1760
工银瑞信 33
1824
交银施罗德 34
587
中信保诚 35
449
建信 36
553
汇丰晋信 37
333
中邮 38
132
信达澳银 39
310
中欧 40
184
浦银安盛 41
675
农银汇理 42
2832
民生加银 43
498
方正富邦 44
196
华宸未来 45
80
华润元大 46
166
中加 47
86
国开泰富 48
549
永赢 49
57
圆信永丰 50
79
恒生前海 51
78
富达 52
98
瑞银 53
150
英仕曼 54
0
富敦 55
7
惠理 56
19
景顺纵横 57
42
路博迈 58
18
安本 59
81
施罗德 60
455
贝莱德 61
4
安中 62
163
桥水 63
2
元胜 64
1
毕盛 65
0
瀚亚 66
16
未来益财 67
20
联博汇智 68
74
安联 69
74
德劭 70
3
罗素 71
0
弘收 72
0
威廉欧奈尔 73
3
鲍尔赛嘉 74
2
迈德瑞 75
4
柏基 76
539
首奕 77
0
韩华 78
62
润晖 79
0
荷宝 80
5
澳帝桦 81
0


In [235]:
# Use "瑞银" as an example to examine the differences in output
# Method 1: Check levenshtein_ratio and use regular expression to see if ShareHolderName contains institution name  

name = '瑞银'
count = 0
    
for j in range(len(df_test)):    
    shareholdername = df_test.iloc[j]['ShareHolderName']
    temp = levenshtein_ratio_and_distance(name, shareholdername, ratio_calc = True)
    
    if temp > 0.2:
        if re.search(name, shareholdername):
            print('index: ' + str(j))
            print('levenshtein ratio: ' + str(temp))
            print(shareholdername)
            count += 1
print(count)

index: 3657
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 8892
levenshtein ratio: 0.21052631578947367
国投瑞银优化增强债券型证券投资基金
index: 8903
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 8914
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 20124
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20204
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20209
levenshtein ratio: 0.25
国投瑞银景气行业证券投资基金
index: 20630
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20665
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20690
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 20701
levenshtein ratio: 0.21052631578947367
国投瑞银创新动力股票型证券投资基金
index: 20720
levenshtein ratio: 0.25
国投瑞银景气行业证券投资基金
index: 20748
levenshtein ratio: 0.21052631578947367
国投瑞银核心企业股票型证券投资基金
index: 21266
levenshtein ratio: 0.21052631578947367
国投瑞银创新动力股票型证券投资基金
index: 22399
levenshtein ratio: 0.21052631578947367
国投瑞银双债增利债券型证券投资基

In [233]:
# Method 2: Only check if ShareHolderName contains institution name

name = '瑞银' 
count = 0
    
for j in range(len(df_test)):
    shareholdername = df_test.iloc[j]['ShareHolderName']
    if re.search(name, shareholdername):
        print('index: ' + str(j))
        print(shareholdername)
        count += 1
print(count)  

index: 608
国投瑞银国家安全灵活配置混合型证券投资基金
index: 636
国投瑞银国家安全灵活配置混合型证券投资基金
index: 664
国投瑞银国家安全灵活配置混合型证券投资基金
index: 697
国投瑞银国家安全灵活配置混合型证券投资基金
index: 713
国投瑞银国家安全灵活配置混合型证券投资基金
index: 741
国投瑞银国家安全灵活配置混合型证券投资基金
index: 764
国投瑞银国家安全灵活配置混合型证券投资基金
index: 772
国投瑞银国家安全灵活配置混合型证券投资基金
index: 793
国投瑞银国家安全灵活配置混合型证券投资基金
index: 2554
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2579
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2642
国投瑞银中证上游资源产业指数证券投资基金(LOF)
index: 2658
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2677
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2754
国投瑞银中证上游资源产业指数证券投资基金(LOF)
index: 2773
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2799
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2861
国投瑞银中证上游资源产业指数证券投资基金(LOF)
index: 2862
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2917
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2987
国投瑞银中证上游资源产业指数证券投资基金(LOF)
index: 2988
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 2991
国投瑞银中证500指数量化增强型证券投资基金
index: 3052
国投瑞银新丝路灵活配置混合型证券投资基金(LOF)
index: 3119
国投瑞银中证500指数量化增强型证券投资基金
index: 3131
国投瑞银中证上游资源产业指数证券投资基金(LOF)
index: 3640
国投瑞银新兴产业混合型证券投资基金(LOF)
index: 3657
国投瑞银成长优选股票型证券投资基

In [240]:
# Method 3: Only check levenshtein_ratio

name = '瑞银'
count = 0

for i in range(len(df_test)):
    shareholdername = df_test.iloc[i]['ShareHolderName'] 
    temp = levenshtein_ratio_and_distance(name, shareholdername, ratio_calc = True)
    
    if temp > 0.2:
        print('index: ' + str(i))
        print('levenshtein ratio: ' + str(temp))
        print(shareholdername)
        count += 1
print(count)

index: 1128
levenshtein ratio: 0.25
挪威中央银行
index: 1152
levenshtein ratio: 0.25
挪威中央银行
index: 1159
levenshtein ratio: 0.25
挪威中央银行
index: 1184
levenshtein ratio: 0.25
挪威中央银行
index: 3657
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 4648
levenshtein ratio: 0.25
挪威中央银行
index: 4656
levenshtein ratio: 0.25
挪威中央银行
index: 4699
levenshtein ratio: 0.25
挪威中央银行
index: 8892
levenshtein ratio: 0.21052631578947367
国投瑞银优化增强债券型证券投资基金
index: 8903
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 8914
levenshtein ratio: 0.21052631578947367
国投瑞银成长优选股票型证券投资基金
index: 18987
levenshtein ratio: 0.25
东方汇理银行
index: 19080
levenshtein ratio: 0.25
法国兴业银行
index: 20124
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20204
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20209
levenshtein ratio: 0.25
国投瑞银景气行业证券投资基金
index: 20630
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 20665
levenshtein ratio: 0.23529411764705882
国投瑞银融华债券型证券投资基金
index: 2069